In [1]:
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import os

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


In [2]:
label_names = [
    '0000000000',
    '0000000001',
    '0000000010',
    '0000000100',
    '0000001000',
    '0000010000',
    '0000100000',
    '0001000000',
    '0010000000',
    '0100000000',
    '1000000000'
]

In [3]:
ImagePATH = f"images/"
categories = glob(ImagePATH+"*")

In [4]:
filedf = pd.DataFrame()
for cat in categories:
    files = glob(cat+"/*")
    tempdf = pd.DataFrame({'filepath':files,'category':cat.split("/")[-1]})
    filedf = pd.concat([filedf,tempdf])
filedf.head()

,filepath,category
0,images/0000000001/image-2021-05-06_12-04-42_16...,0000000001
1,images/0000000001/image-2021-05-06_11-57-25_16...,0000000001
2,images/0000000001/image-2021-05-06_03-17-04_16...,0000000001
3,images/0000000001/image-2021-05-06_03-05-24_16...,0000000001
4,images/0000000001/image-2021-05-06_02-53-03_16...,0000000001


In [5]:
gby_cnt = filedf.groupby("category").aggregate('count').rename(columns = {'filepath':'cnt'}).reset_index().sort_values(by='cnt',ascending=False)


In [6]:
focus_cats = gby_cnt[gby_cnt['cnt'] >=40]['category'].values
filedf  = filedf[filedf['category'].isin(focus_cats)]
filedf.category.value_counts()

0000000000    960
0000000010    960
0100000000    960
1000000000    960
0000100000    960
0001000000    960
0010000000    960
0000001000    960
0000000100    960
0000000001    960
0000010000    920
Name: category, dtype: int64

In [7]:
X_train, X_test, _, _ = train_test_split(
        filedf, filedf['category'],stratify=filedf['category'], test_size=0.4)

X_test, X_val, _, _ = train_test_split(
        X_test, X_test['category'], stratify=X_test['category'], test_size=0.5)

X_train

,filepath,category
785,images/0010000000/image-2021-05-06_02-51-02_16...,0010000000
726,images/0000000100/image-2021-05-06_11-58-58_16...,0000000100
261,images/0000000001/image-2021-05-06_03-14-41_16...,0000000001
620,images/0000000001/image-2021-05-06_11-53-37_16...,0000000001
737,images/0000000000/image-2021-05-06_12-03-06_16...,0000000000
...,...,...
657,images/0001000000/image-2021-05-06_12-09-29_16...,0001000000
350,images/1000000000/image-2021-05-06_02-52-41_16...,1000000000
325,images/0000010000/image-2021-05-06_02-46-26_16...,0000010000
605,images/0000000000/image-2021-05-06_12-08-47_16...,0000000000


In [8]:
X_train['type'] = 'train'
X_val['type'] = 'val'
X_test['type'] = 'test'

<ipython-input-8-27ac73f13727>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['type'] = 'train'


In [9]:
fulldf = pd.concat([X_train,X_test,X_val])
fulldf.type.value_counts()

train    6312
val      2104
test     2104
Name: type, dtype: int64

In [10]:
fulldf.head()

,filepath,category,type
785,images/0010000000/image-2021-05-06_02-51-02_16...,0010000000,train
726,images/0000000100/image-2021-05-06_11-58-58_16...,0000000100,train
261,images/0000000001/image-2021-05-06_03-14-41_16...,0000000001,train
620,images/0000000001/image-2021-05-06_11-53-37_16...,0000000001,train
737,images/0000000000/image-2021-05-06_12-03-06_16...,0000000000,train


In [11]:
fulldf.category.value_counts()[0]/len(fulldf)


0.09125475285171103

In [12]:
!mkdir data
!rm -rf data/
!rm -rf data/train
!rm -rf data/test
!rm -rf data/val
!mkdir data/
!mkdir data/train
!mkdir data/test
!mkdir data/val

In [13]:
fulldf.category.unique()


array(['0010000000', '0000000100', '0000000001', '0000000000',
       '0000001000', '0000000010', '1000000000', '0001000000',
       '0100000000', '0000100000', '0000010000'], dtype=object)

In [18]:
for cat in fulldf.category.unique():
    os.system(f"mkdir data/train/'{cat}'") 
    os.system(f"mkdir data/test/'{cat}'") 
    os.system(f"mkdir data/val/'{cat}'")
fulldf.head()

fulldf.shape
fulldf.to_csv('file_name.csv')

In [15]:
for i,row in fulldf.iterrows():
    # Boat category
    cat = row['category']
    # section is train,val or test
    section = row['type']
    # input filepath to copy
    ipath = row['filepath']
    # output filepath to paste
    opath = ipath.replace(f'images/',f'data/{section}/')
    print(opath)
    # running the cp command
    os.system(f"cp '{ipath}' '{opath}'")

data/train/0010000000/image-2021-05-06_02-51-02_1620287462036.jpg
data/train/0000000100/image-2021-05-06_11-58-58_1620320338690.jpg
data/train/0000000001/image-2021-05-06_03-14-41_1620288881455.jpg
data/train/0000000001/image-2021-05-06_11-53-37_1620320017824.jpg
data/train/0000000000/image-2021-05-06_12-03-06_1620320586125.jpg
data/train/0000000001/image-2021-05-06_11-58-47_1620320327967.jpg
data/train/0000001000/image-2021-05-06_02-53-23_1620287603134.jpg
data/train/0000000010/image-2021-05-06_03-11-37_1620288697372.jpg
data/train/0000000010/image-2021-05-06_12-30-05_1620322205706.jpg
data/train/0000001000/image-2021-05-06_02-50-32_1620287432650.jpg
data/train/0000000000/image-2021-05-06_02-51-33_1620287493979.jpg
data/train/1000000000/image-2021-05-06_12-28-02_1620322082474.jpg
data/train/0000000000/image-2021-05-06_03-11-30_1620288690662.jpg
data/train/0000000100/image-2021-05-06_12-18-30_1620321510786.jpg
data/train/0000000100/image-2021-05-06_12-27-33_1620322053169.jpg
data/train

data/train/0000010000/image-2021-05-06_02-45-06_1620287106531.jpg
data/train/0000000100/image-2021-05-06_02-44-53_1620287093860.jpg
data/train/0100000000/image-2021-05-06_02-46-55_1620287215657.jpg
data/train/1000000000/image-2021-05-06_03-17-37_1620289057112.jpg
data/train/0000000001/image-2021-05-06_03-04-13_1620288253949.jpg
data/train/1000000000/image-2021-05-06_12-30-45_1620322245615.jpg
data/train/0010000000/image-2021-05-06_12-32-09_1620322329731.jpg
data/train/0000000100/image-2021-05-06_12-31-06_1620322266639.jpg
data/train/0000100000/image-2021-05-06_02-57-20_1620287840147.jpg
data/train/1000000000/image-2021-05-06_12-31-31_1620322291576.jpg
data/train/0000001000/image-2021-05-06_02-57-08_1620287828481.jpg
data/train/0010000000/image-2021-05-06_02-53-53_1620287633111.jpg
data/train/0000000100/image-2021-05-06_03-14-04_1620288844308.jpg
data/train/1000000000/image-2021-05-06_11-54-32_1620320072998.jpg
data/train/0000010000/image-2021-05-06_12-31-12_1620322272001.jpg
data/train

data/train/0000000100/image-2021-05-06_12-30-12_1620322212315.jpg
data/train/0000000100/image-2021-05-06_03-12-26_1620288746957.jpg
data/train/1000000000/image-2021-05-06_12-11-04_1620321064026.jpg
data/train/0000000000/image-2021-05-06_02-40-42_1620286842361.jpg
data/train/0010000000/image-2021-05-06_12-26-13_1620321973400.jpg
data/train/0000000000/image-2021-05-06_12-07-28_1620320848175.jpg
data/train/0001000000/image-2021-05-06_02-55-00_1620287700518.jpg
data/train/0001000000/image-2021-05-06_12-02-27_1620320547187.jpg
data/train/0010000000/image-2021-05-06_02-45-24_1620287124270.jpg
data/train/0000000010/image-2021-05-06_12-01-45_1620320505973.jpg
data/train/0100000000/image-2021-05-06_02-41-34_1620286894174.jpg
data/train/0000000100/image-2021-05-06_03-16-28_1620288988431.jpg
data/train/0000100000/image-2021-05-06_02-46-35_1620287195883.jpg
data/train/0000000000/image-2021-05-06_12-10-01_1620321001157.jpg
data/train/0000010000/image-2021-05-06_02-57-12_1620287832065.jpg
data/train

data/train/0000100000/image-2021-05-06_03-05-51_1620288351969.jpg
data/train/0000000100/image-2021-05-06_02-47-36_1620287256059.jpg
data/train/0000000001/image-2021-05-06_12-03-12_1620320592653.jpg
data/train/0000001000/image-2021-05-06_12-19-53_1620321593623.jpg
data/train/0000000010/image-2021-05-06_12-08-59_1620320939819.jpg
data/train/0000000010/image-2021-05-06_02-53-09_1620287589045.jpg
data/train/0000000001/image-2021-05-06_12-16-31_1620321391086.jpg
data/train/0000000000/image-2021-05-06_02-56-44_1620287804211.jpg
data/train/0000000001/image-2021-05-06_03-11-33_1620288693358.jpg
data/train/0000100000/image-2021-05-06_12-00-51_1620320451362.jpg
data/train/0000000001/image-2021-05-06_12-25-49_1620321949753.jpg
data/train/0000010000/image-2021-05-06_12-00-43_1620320443340.jpg
data/train/0000010000/image-2021-05-06_12-31-59_1620322319882.jpg
data/train/0001000000/image-2021-05-06_12-02-24_1620320544972.jpg
data/train/0000000100/image-2021-05-06_12-29-20_1620322160631.jpg
data/train

data/train/0000000100/image-2021-05-06_12-03-24_1620320604235.jpg
data/train/0000000010/image-2021-05-06_12-28-19_1620322099640.jpg
data/train/0000000010/image-2021-05-06_11-58-54_1620320334730.jpg
data/train/1000000000/image-2021-05-06_02-54-05_1620287645918.jpg
data/train/0000000100/image-2021-05-06_11-55-04_1620320104458.jpg
data/train/0000100000/image-2021-05-06_02-50-48_1620287448355.jpg
data/train/1000000000/image-2021-05-06_12-31-30_1620322290240.jpg
data/train/0010000000/image-2021-05-06_02-49-35_1620287375345.jpg
data/train/0000001000/image-2021-05-06_12-29-22_1620322162674.jpg
data/train/0010000000/image-2021-05-06_12-05-33_1620320733085.jpg
data/train/0001000000/image-2021-05-06_02-55-00_1620287700135.jpg
data/train/0001000000/image-2021-05-06_02-55-02_1620287702623.jpg
data/train/0000100000/image-2021-05-06_11-56-41_1620320201206.jpg
data/train/0000000001/image-2021-05-06_12-28-16_1620322096951.jpg
data/train/0000010000/image-2021-05-06_02-54-48_1620287688994.jpg
data/train

...

data/val/0001000000/image-2021-05-06_12-28-38_1620322118068.jpg
data/val/0000000010/image-2021-05-06_03-13-13_1620288793181.jpg
data/val/0001000000/image-2021-05-06_11-59-30_1620320370299.jpg
data/val/0000000100/image-2021-05-06_02-46-12_1620287172025.jpg
data/val/0000000001/image-2021-05-06_02-45-57_1620287157908.jpg
data/val/0000000010/image-2021-05-06_03-04-19_1620288259595.jpg
data/val/0100000000/image-2021-05-06_12-30-41_1620322241739.jpg
data/val/0000000100/image-2021-05-06_03-13-16_1620288796496.jpg
data/val/0001000000/image-2021-05-06_12-26-11_1620321971579.jpg
data/val/0000001000/image-2021-05-06_11-55-11_1620320111421.jpg
data/val/0000001000/image-2021-05-06_12-29-23_1620322163703.jpg
data/val/0000000010/image-2021-05-06_02-50-19_1620287419709.jpg
data/val/0000001000/image-2021-05-06_03-15-40_1620288940073.jpg
data/val/0000100000/image-2021-05-06_02-42-32_1620286952234.jpg
data/val/0010000000/image-2021-05-06_02-44-00_1620287040267.jpg
data/val/0000000010/image-2021-05-06_12-

data/val/0000000000/image-2021-05-06_12-28-13_1620322093030.jpg
data/val/0010000000/image-2021-05-06_02-44-05_1620287045521.jpg
data/val/0000000010/image-2021-05-06_12-07-38_1620320858925.jpg
data/val/0010000000/image-2021-05-06_12-09-38_1620320978627.jpg
data/val/0001000000/image-2021-05-06_03-17-25_1620289045639.jpg
data/val/0000100000/image-2021-05-06_12-29-29_1620322169840.jpg
data/val/0010000000/image-2021-05-06_12-17-31_1620321451703.jpg
data/val/0100000000/image-2021-05-06_02-46-54_1620287214937.jpg
data/val/0000010000/image-2021-05-06_12-28-29_1620322109506.jpg
data/val/1000000000/image-2021-05-06_12-28-58_1620322138052.jpg
data/val/0001000000/image-2021-05-06_02-56-11_1620287771645.jpg
data/val/0010000000/image-2021-05-06_12-31-23_1620322283524.jpg
data/val/1000000000/image-2021-05-06_11-58-19_1620320299325.jpg
data/val/0000010000/image-2021-05-06_11-56-34_1620320194140.jpg
data/val/1000000000/image-2021-05-06_03-05-04_1620288304046.jpg
data/val/1000000000/image-2021-05-06_12-

data/val/0000000000/image-2021-05-06_11-54-47_1620320087052.jpg
data/val/0000000100/image-2021-05-06_12-09-06_1620320946271.jpg
data/val/0000100000/image-2021-05-06_03-15-00_1620288900791.jpg
data/val/0000000100/image-2021-05-06_11-57-39_1620320259188.jpg
data/val/0000100000/image-2021-05-06_03-13-26_1620288806586.jpg
data/val/0000100000/image-2021-05-06_12-02-20_1620320540945.jpg
data/val/0010000000/image-2021-05-06_02-51-01_1620287461118.jpg
data/val/0000000010/image-2021-05-06_03-11-35_1620288695955.jpg
data/val/0000000001/image-2021-05-06_02-40-45_1620286845052.jpg
data/val/0000000010/image-2021-05-06_12-19-42_1620321582361.jpg
data/val/1000000000/image-2021-05-06_12-28-57_1620322137622.jpg
data/val/0000100000/image-2021-05-06_03-15-48_1620288948598.jpg
data/val/0100000000/image-2021-05-06_12-09-42_1620320982518.jpg
data/val/0000000100/image-2021-05-06_02-48-59_1620287339932.jpg
data/val/0010000000/image-2021-05-06_02-52-27_1620287547276.jpg
data/val/0001000000/image-2021-05-06_03-

data/val/0100000000/image-2021-05-06_02-46-54_1620287214004.jpg
data/val/0010000000/image-2021-05-06_12-07-01_1620320821233.jpg
data/val/0000000001/image-2021-05-06_03-12-18_1620288738900.jpg
data/val/0000000100/image-2021-05-06_12-27-34_1620322054215.jpg
data/val/0000010000/image-2021-05-06_02-53-29_1620287609061.jpg
data/val/0100000000/image-2021-05-06_03-15-14_1620288914741.jpg
data/val/1000000000/image-2021-05-06_12-31-29_1620322289579.jpg
data/val/0000000010/image-2021-05-06_11-56-14_1620320174395.jpg
data/val/0000000100/image-2021-05-06_03-14-02_1620288842274.jpg
data/val/0100000000/image-2021-05-06_11-55-40_1620320140780.jpg
data/val/0010000000/image-2021-05-06_12-05-34_1620320734090.jpg
data/val/0000000010/image-2021-05-06_12-31-48_1620322308869.jpg
data/val/0100000000/image-2021-05-06_02-56-25_1620287785675.jpg
data/val/0100000000/image-2021-05-06_03-13-38_1620288818872.jpg
data/val/0000000001/image-2021-05-06_02-44-40_1620287080029.jpg
data/val/0000000000/image-2021-05-06_11-

data/val/0000001000/image-2021-05-06_12-28-26_1620322106026.jpg
data/val/0100000000/image-2021-05-06_12-20-26_1620321626187.jpg
data/val/1000000000/image-2021-05-06_11-54-30_1620320070562.jpg
data/val/0000010000/image-2021-05-06_03-04-36_1620288276174.jpg
data/val/0000001000/image-2021-05-06_12-26-52_1620322012112.jpg
data/val/0001000000/image-2021-05-06_02-48-02_1620287282612.jpg
data/val/0010000000/image-2021-05-06_02-44-02_1620287042448.jpg
data/val/0000000100/image-2021-05-06_03-14-47_1620288887361.jpg
data/val/1000000000/image-2021-05-06_02-45-35_1620287135733.jpg
data/val/0100000000/image-2021-05-06_12-19-09_1620321549586.jpg
data/val/0010000000/image-2021-05-06_12-32-10_1620322330712.jpg
data/val/0000001000/image-2021-05-06_11-56-28_1620320188197.jpg
data/val/0000000001/image-2021-05-06_12-19-37_1620321577445.jpg
data/val/0000001000/image-2021-05-06_03-15-42_1620288942068.jpg
data/val/0001000000/image-2021-05-06_03-14-16_1620288856494.jpg
data/val/0010000000/image-2021-05-06_03-